In [43]:
# Import packages for building map
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
pbk.output_notebook()
from sodapy import Socrata
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes 
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource, CustomJS,
                          CustomJSFilter, GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

Loading BokehJS ...

In [44]:
# Load file with state geometry
contiguous_cat = gpd.read_file('divisions-administratives-v1r0-comarques-1000000-20210122.shp')
cols = contiguous_cat.select_dtypes(include=[np.object]).columns
contiguous_cat[cols] = contiguous_cat[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
# Load file with each region's population
df_pobl = pd.read_excel("PoblacioComarques.xls")
# importar el nostre dataset
client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16774)
# Convert to pandas DataFrame
results_df =  gpd.GeoDataFrame.from_records(results)
contiguous_cat

/var/folders/tc/wsbp34qn42s_gdgnmlhtt4480000gn/T/ipykernel_35125/474452583.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = contiguous_cat.select_dtypes(include=[np.object]).columns


,CODICOMAR,NOMCOMAR,geometry
0,01,Alt Camp,"POLYGON ((373020.449 4590188.741, 372482.744 4..."
1,02,Alt Emporda,"MULTIPOLYGON (((513760.140 4662388.220, 513758..."
2,03,Alt Penedes,"POLYGON ((386593.300 4595896.200, 388616.100 4..."
3,04,Alt Urgell,"MULTIPOLYGON (((360800.100 4649174.000, 360624..."
4,05,Alta Ribagorca,"POLYGON ((327485.124 4721105.796, 327436.728 4..."
5,06,Anoia,"POLYGON ((371426.378 4628557.172, 372106.708 4..."
6,07,Bages,"POLYGON ((417259.067 4646530.811, 417513.903 4..."
7,08,Baix Camp,"MULTIPOLYGON (((334244.690 4546609.460, 334242..."
8,09,Baix Ebre,"MULTIPOLYGON (((311847.200 4514855.880, 311833..."
9,10,Baix Emporda,"MULTIPOLYGON (((498479.410 4622829.390, 498478..."


In [45]:
# num d'accidents en funció de la comarca i ordenats alfabèticament respecte del nom de la comarca
com = pd.value_counts(results_df['nomcom'])
com = com.to_frame() # convertir a Data Frame
com = com.reset_index() # introduir un index (0, 1, 2...)
com = com.sort_values("index") # la columna de velocitat es diu index, ordenar-la
com = com.rename(columns={"index": "nomcom", "nomcom": "#accidents"}) # canviar els noms correctes
Comar = gpd.GeoDataFrame(com).set_index("nomcom")
Comar = Comar.reset_index()
Comar.head() #42

,nomcom,#accidents
0,Alt Camp,146
1,Alt Emporda,563
2,Alt Penedes,397
3,Alt Urgell,187
4,Alta Ribagorca,30


In [46]:
prov = results_df[['nomcom', 'nomdem']]
prov = prov.sort_values('nomcom') 
prov.head() #16mil

,nomcom,nomdem
214,Alt Camp,Tarragona
1054,Alt Camp,Tarragona
8587,Alt Camp,Tarragona
14204,Alt Camp,Tarragona
6112,Alt Camp,Tarragona


In [47]:
merge = prov.merge(Comar, left_on='nomcom', right_on='nomcom') # associem la comarca a la província, merge 16 mil
#filtrar Bcn
merge.set_index('nomdem', inplace=True)
merge_bcn=merge.loc[['Tarragona']]
#eliminar duplicats
merge_bcn = merge_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn=merge_bcn.reset_index()
bcn.head() #13


,nomdem,nomcom,#accidents
0,Tarragona,Alt Camp,146
1,Tarragona,Baix Camp,485
2,Tarragona,Baix Ebre,297
3,Tarragona,Baix Penedes,271
4,Tarragona,Conca de Barbera,69


In [48]:
#contiguous_cat = contiguous_cat.drop({0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41})
contiguous_cat = contiguous_cat.drop({1,2,3,4,5,6,9,10,12,13,14,16,17,18,19,20,22,23,24,25,26,27,30,31,32,33,34,37,38,39,40,41})


In [49]:
geom = contiguous_cat[["NOMCOMAR", "geometry"]] # escollir les columnes: comarca i geometria de mcp
geom = geom.sort_values('NOMCOMAR') # ordenar alfabèticament
Geom = gpd.GeoDataFrame(geom).set_index("NOMCOMAR") # tornem a posar l'índex
Geom = Geom.reset_index()
Geom = Geom.rename(columns={"NOMCOMAR": "nomcom"}) # canviar el nom
Geom.head() #13

,nomcom,geometry
0,Alt Camp,"POLYGON ((373020.449 4590188.741, 372482.744 4..."
1,Baix Camp,"MULTIPOLYGON (((334244.690 4546609.460, 334242..."
2,Baix Ebre,"MULTIPOLYGON (((311847.200 4514855.880, 311833..."
3,Baix Penedes,"MULTIPOLYGON (((385851.064 4574022.148, 385800..."
4,Conca de Barbera,"POLYGON ((369548.800 4603947.800, 370045.800 4..."


In [50]:
merge2 = prov.merge(Geom, left_on='nomcom', right_on='nomcom') # associem la comarca a la província
#filtrar Bcn
merge2.set_index('nomdem', inplace=True)
merge2_bcn=merge.loc[['Tarragona']]
#eliminar duplicats
merge2_bcn = merge2_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn2=merge_bcn.reset_index()
bcn2.head() #13

,nomdem,nomcom,#accidents
0,Tarragona,Alt Camp,146
1,Tarragona,Baix Camp,485
2,Tarragona,Baix Ebre,297
3,Tarragona,Baix Penedes,271
4,Tarragona,Conca de Barbera,69


In [51]:
# seleccionar columna de geometries
geometry = Geom[['geometry']]
# ordenem alfabèticament i deixem bonic el dataset de poblacions
df_pobl=df_pobl.sort_values('comarca')
df_pobl=pd.DataFrame(df_pobl).set_index('comarca')


In [52]:
df_pobl=df_pobl.reset_index()
df_pobl.head() #42

,comarca,pobl,pobl_percent
0,Alt Camp,44168,0.0059
1,Alt Emporda,136981,0.0182
2,Alt Penedes,107724,0.0143
3,Alt Urgell,20201,0.0027
4,Alta Ribagorca,3813,0.0005


In [53]:
merge_norm = prov.merge(df_pobl, left_on='nomcom', right_on='comarca') # associem la comarca a la província
#filtrar Bcn
merge_norm.set_index('nomdem', inplace=True)
merge_bcn_norm=merge_norm.loc[['Tarragona']]
#eliminar duplicats
merge_bcn_norm = merge_bcn_norm.drop_duplicates(subset='nomcom',keep='first')
bcn_norm = merge_bcn_norm.reset_index()
bcn_norm.head() #13

,nomdem,nomcom,comarca,pobl,pobl_percent
0,Tarragona,Alt Camp,Alt Camp,44168,0.0059
1,Tarragona,Baix Camp,Baix Camp,190004,0.0252
2,Tarragona,Baix Ebre,Baix Ebre,77170,0.0102
3,Tarragona,Baix Penedes,Baix Penedes,101979,0.0135
4,Tarragona,Conca de Barbera,Conca de Barbera,20013,0.0027


In [54]:
# seleccionar columnes pobl i pobl_percent
norm = bcn_norm[['pobl', 'pobl_percent']]
combo = pd.concat([bcn, geometry, norm], axis=1) # juntem comarques, #accidents, geometria i poblacions de bcn
combo['accpercadamilhab_tgn'] = combo['#accidents']/combo['pobl']*1000 # afegim col per a #accidents per cada 1000 habitants
combo.head()


,nomdem,nomcom,#accidents,geometry,pobl,pobl_percent,accpercadamilhab_tgn
0,Tarragona,Alt Camp,146,"POLYGON ((373020.449 4590188.741, 372482.744 4...",44168,0.0059,3.305561
1,Tarragona,Baix Camp,485,"MULTIPOLYGON (((334244.690 4546609.460, 334242...",190004,0.0252,2.552578
2,Tarragona,Baix Ebre,297,"MULTIPOLYGON (((311847.200 4514855.880, 311833...",77170,0.0102,3.848646
3,Tarragona,Baix Penedes,271,"MULTIPOLYGON (((385851.064 4574022.148, 385800...",101979,0.0135,2.657410
4,Tarragona,Conca de Barbera,69,"POLYGON ((369548.800 4603947.800, 370045.800 4...",20013,0.0027,3.447759


In [55]:
#triar columnes
combo = combo[['#accidents', 'nomcom', 'geometry', 'accpercadamilhab_tgn']]
combo

,#accidents,nomcom,geometry,accpercadamilhab_tgn
0,146,Alt Camp,"POLYGON ((373020.449 4590188.741, 372482.744 4...",3.305561
1,485,Baix Camp,"MULTIPOLYGON (((334244.690 4546609.460, 334242...",2.552578
2,297,Baix Ebre,"MULTIPOLYGON (((311847.200 4514855.880, 311833...",3.848646
3,271,Baix Penedes,"MULTIPOLYGON (((385851.064 4574022.148, 385800...",2.657410
4,69,Conca de Barbera,"POLYGON ((369548.800 4603947.800, 370045.800 4...",3.447759
5,219,Montsia,"MULTIPOLYGON (((300644.450 4494148.850, 300643...",3.226519
6,49,Priorat,"POLYGON ((320525.200 4579659.500, 321213.700 4...",5.313957
7,109,Ribera d'Ebre,"POLYGON ((290923.800 4573932.800, 291056.392 4...",5.017954
8,543,Tarragones,"POLYGON ((362512.057 4568174.143, 363125.922 4...",2.141319
9,54,Terra Alta,"POLYGON ((280206.273 4568311.294, 280325.826 4...",4.741417


In [56]:
combo = gpd.GeoDataFrame(combo, geometry=combo['geometry'])
type(combo)

geopandas.geodataframe.GeoDataFrame

In [57]:
# Representacio interactiva
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = combo.to_json())

In [58]:
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '50': '500',
               '1000':'10000', '15000':'150000',
               '20000':'20000', '25000':'250000',
               '30000':'30000', '350000':'350000',
               '40000':'40000+'}

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = "Nombre d'accidents per cada 1000 habitants 2010-18", plot_height = 600 ,
           plot_width = 600, toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'accpercadamilhab_tgn', 'transform' : color_mapper},
                   line_color = 'gray', line_width = 0.45, fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Comarca','@nomcom'),('# Accidents', '@accpercadamilhab_tgn')]))

# Specify layout
p.add_layout(color_bar, 'below')

show(p)